In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
cd '../gdrive/MyDrive/SSAC/3조'

/gdrive/.shortcut-targets-by-id/15_BxZVEQYCdGCGiQ5nexpWPc1cgHVe4w/3조


In [ ]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic') 

# 게시글 데이터, CTR 예측결과 불러오기

In [ ]:
df = pd.read_csv('data/텍스트유사도.csv')
ctr = pd.read_csv('data/의류/total_ctr.csv')

In [ ]:
df = df.set_index('content_id')
ctr= ctr.set_index('content_id')
ctr = ctr.loc[df.index]

df_img = df[df.bg_rgb != '-1']
ctr_img = ctr.loc[df_img.index]

- 게시글 데이터에 전체 CTR 평균, 성별 평균 컬럼 추가

In [ ]:
gender_ctr = ctr.groupby(['gender','content_id']).mean()
df['ctr_man'] = gender_ctr.loc[1]['ctr']
df['ctr_woman'] = gender_ctr.loc[2]['ctr']
df['ctr'] = ctr.groupby('content_id').mean()['ctr']

gender_ctr_img = ctr_img.groupby(['gender','content_id']).mean()
df_img['ctr_man'] = gender_ctr_img.loc[1]['ctr']
df_img['ctr_woman'] = gender_ctr_img.loc[2]['ctr']
df_img['ctr'] = ctr_img.groupby('content_id').mean()['ctr']

### 전체 게시글 CTR 평균

In [ ]:
df[['ctr', 'ctr_man', 'ctr_woman']].mean()

ctr          0.021815
ctr_man      0.023107
ctr_woman    0.020523
dtype: float64

### 의류 대분류(남성의류, 여성의류)

In [ ]:
man_df = df[df['content_cat_1'] == '남성의류']
woman_df = df[df['content_cat_1'] == '여성의류']

In [ ]:
man_df[['ctr', 'ctr_man', 'ctr_woman']].mean()

ctr          0.024014
ctr_man      0.027366
ctr_woman    0.020662
dtype: float64

In [ ]:
woman_df[['ctr', 'ctr_man', 'ctr_woman']].mean()

ctr          0.020062
ctr_man      0.019712
ctr_woman    0.020413
dtype: float64

### 거래 지역

In [ ]:
place_df = df.groupby('content_place').mean()[['ctr', 'ctr_man', 'ctr_woman']]

In [ ]:
plt.plot(place_df['ctr'])

### 데이터 비교하는 함수
- threshold 기준으로 높은 게시글들과 낮은 게시글들간의 평균 CTR이 얼마나 차이나는지 보여주는 함수

In [ ]:

def printer(col, threshold, df=df,ctr=ctr):
    '''
    해당 컬럼 값이 threshoid보다 큰 데이터들과 작은 데이터들의 ctr 평균을 보여주는 함수
    '''
    print(f"{col} | {threshold}보다 큰거 : {round(ctr.loc[df[col]>threshold].mean()['ctr']*100,2)} %")
    print(f"{col} | {threshold}보다 작은거 : {round(ctr.loc[df[col]<=threshold].mean()['ctr']*100,2)} %")
    print('-'*50)

def classifier(col, threshold, df, ctr):
    '''
    해당 컬럼 값이 threshoid보다 큰 데이터들과 작은 데이터들의 ctr 평균을 반환하는 함수
    '''
    low = round(ctr.loc[df[col]<=threshold].median()['ctr']*100,2)
    high = round(ctr.loc[df[col]>threshold].median()['ctr']*100,2)
    return low, high

# 위에서 계산한 두 값간의 차이를 계산하는 함수
def calculate_diff(low, high):
    '''
    위에서 계산한 두 값간의 차이를 계산하는 함수
    '''
    if low > high:
        status = 'low'
        ratio = round((low - high)/high*100,2)
    else:
        status = 'high'
        ratio = round((high - low)/low*100,2)
    return status, ratio

def analysis_features(classify_features):
    '''
    각 컬럼마다 해당 threshold 값을 기준으로 높은 게시글들과 낮은 게시글들 중 어느 쪽의 평균 CTR이 얼마나 높은지 df형식으로 반환하는 함수
    '''
    result_df = pd.DataFrame(columns=['데이터', '컬럼', 'threshold', '구분', 'CTR'])
    diff_df = pd.DataFrame(columns=['데이터', '컬럼', 'threshold', '구분', 'ratio'])
    for i, (col, threshold) in enumerate(classify_features.items()):
        low, high = classifier(col, threshold, df, ctr)
        result_df.loc[2*i] = {'데이터': '전체', '컬럼': col, 'threshold': threshold, '구분': 'low', 'CTR': low}
        result_df.loc[2*i+1] = {'데이터': '전체', '컬럼': col, 'threshold': threshold, '구분': 'high', 'CTR': high}
        status, ratio = calculate_diff(low, high)
        diff_df.loc[i] = {'데이터': '전체', '컬럼': col, 'threshold': threshold, '구분': status, 'ratio': ratio}

    return result_df, diff_df

- 위 함수 기반으로 게시글들 비교

In [ ]:
threshold_dict = {
    # 컨텐츠
    'content_price': 50000,
    'content_delivery_fee': 0,
    'content_b_pay': 0,
    'qty': 1,
    'is_ad': 0,
    'content_status': 0,
    'content_used': 0,
    'tradable': 0,
    'content_views': 100,
    'content_comment_count': 10,
    'content_likes': 5,

    # 광고주(판매자)
    'adv_item_count': 10,
    'adv_follower_count': 10,
    'adv_review_count': 10,
    'adv_grade': 9,

    # 이미지
    'background_ratio': 0.3,
    'bg_noise': 0.3,
    'nobg_noise': 0.3,
    
    # 키워드
    '제목-키워드': 0.5,
    '제목-카테고리': 0.5,
    '키워드-카테고리': 0.5,
    '제목설명-키워드': 0.5, 
    '제목키워드설명-카테고리': 0.5,
}

In [ ]:
result_df, diff_df = analysis_features(threshold_dict)
diff_df

,데이터,컬럼,threshold,구분,ratio
0,전체,content_price,50000,high,24.26
1,전체,content_delivery_fee,0,low,39.64
2,전체,content_b_pay,0,high,8.51
3,전체,qty,1,high,23.78
4,전체,is_ad,0,high,17.24
5,전체,content_status,0,high,22.92
6,전체,content_used,0,low,12.06
7,전체,tradable,0,high,4.86
8,전체,content_views,100,high,9.35
9,전체,content_comment_count,10,high,3.45
